### Open AI Gym

Reinforcement Learning with CartePole

In [1]:
!pip install gym tqdm pillow

    100% |████████████████████████████████| 112kB 2.4MB/s 
    100% |████████████████████████████████| 61kB 2.5MB/s 
    100% |████████████████████████████████| 1.0MB 686kB/s 
    100% |████████████████████████████████| 829kB 542kB/s 
  Running setup.py install for future ... - \ | / - \ | / - \ | / done
  Running setup.py install for gym ... - \ | / - done


In [4]:
!brew install python-opengl

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/science, homebrew/core).
==> New Formulae
amber                      hlint                      plank
augustus                   hmmer                      posh
ballerina                  howdoi                     primer3
bareos-client              jdupes                     qtkeychain
boost-python3              krakend                    restview
calicoctl                  kube-ps1                   shelltestrunner
chrome-export              libbitcoin-consensus       shogun
clblast                    libccd                     siril
console_bridge             libdill                    skafos
container-diff             libjwt                     srt
coreos-ct                  libsbol                    terraforming
dartsim                    mafft                      tmux-xpanes
dashing                    mariadb-connector-odbc     tomcat@8
dynare                     mdcat                      unravel
fcl     

In [ ]:
!brew install cmake boost boost-python sdl2 swig wget

==> Downloading https://homebrew.bintray.com/bottles/cmake-3.10.2.high_sierra.bo
==> Downloading from https://akamai.bintray.com/7a/7ad98f403e21c76cfd0789d83acff
######################################################################## 100.0%
==> Pouring cmake-3.10.2.high_sierra.bottle.tar.gz
==> Caveats
Emacs Lisp files have been installed to:
  /usr/local/share/emacs/site-lisp/cmake
==> Summary
🍺  /usr/local/Cellar/cmake/3.10.2: 2,334 files, 32.4MB
==> Downloading https://homebrew.bintray.com/bottles/boost-1.66.0.high_sierra.bo
==> Downloading from https://akamai.bintray.com/78/78cb090c515e20aa7307c6619a055
######################################################################## 100.0%
==> Pouring boost-1.66.0.high_sierra.bottle.tar.gz
🍺  /usr/local/Cellar/boost/1.66.0: 13,101 files, 435.2MB
==> Downloading https://homebrew.bintray.com/bottles/boost-python-1.66.0_1.high_
==> Downloading from https://akamai.bintray.com/90/908a5484b565b1ee55ccec7d1f3e1
##################################

In [1]:
# Common imports
import numpy as np
import os
import sys
import gym
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import time
from tqdm import tqdm_notebook
import tensorflow as tf
%matplotlib nbagg

ModuleNotFoundError: No module named 'gym'

In [0]:
os.chdir('..')

In [4]:
# Utility function that hides a lot of the heavy lifting

from render_utils import *
from reward_functions import *

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: ignored

In [0]:
# When the import above doesn't work

import numpy as np
import os
import sys
import gym
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import matplotlib
import matplotlib.animation as animation
import numpy as np
from PIL import Image
import time
from PIL import Image, ImageDraw, ImageFont
import tensorflow as tf


# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rl"


def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)


# ----------------------------- Render Functions -----------------------------------


try:
    from pyglet.gl import gl_info

    openai_cart_pole_rendering = True  # no problem, let's use OpenAI gym's rendering function
except Exception:
    openai_cart_pole_rendering = False  # probably no X server available, let's use our own rendering function


def update_scene(num, frames, patch):
    patch.set_data(frames[num])
    return patch,


def plot_animation(frames, repeat=False, interval=40):
    plt.close()  # or else nbagg sometimes plots in the previous cell
    fig = plt.figure()
    patch = plt.imshow(frames[0])
    plt.axis('off')
    return animation.FuncAnimation(fig, update_scene, fargs=(frames, patch), frames=len(frames), repeat=repeat,
                                   interval=interval)


def plot_environment(env, figsize=(5, 4)):
    plt.close()  # or else nbagg sometimes plots in the previous cell
    plt.figure(figsize=figsize)
    img = env.render(mode="rgb_array")
    plt.imshow(img)
    plt.axis("off")
    plt.show()


# ----------------------------- Custom HUD -----------------------------------


def render_cart_pole(env, obs, step, action_val):
    if not openai_cart_pole_rendering:
        # use OpenAI gym's rendering function
        return env.render(mode="rgb_array")
    else:
        # rendering for the cart pole environment (in case OpenAI gym can't do it)
        img_w = 600
        img_h = 400
        cart_w = img_w // 12
        cart_h = img_h // 15
        pole_len = img_h // 3.5
        pole_w = img_w // 80 + 1
        x_width = 2
        max_ang = 0.2
        bg_col = (255, 255, 255)
        cart_col = 0x000000  # Blue Green Red
        pole_col = 0x669acc  # Blue Green Red

        pos, vel, ang, ang_vel = obs
        img = Image.new('RGB', (img_w, img_h), bg_col)
        draw = ImageDraw.Draw(img)
        cart_x = pos * img_w // x_width + img_w // x_width
        cart_y = img_h * 95 // 100
        top_pole_x = cart_x + pole_len * np.sin(ang)
        top_pole_y = cart_y - cart_h // 2 - pole_len * np.cos(ang)
        draw.line((0, cart_y, img_w, cart_y), fill=0)
        draw.line((cart_x, cart_y - cart_h // 2, top_pole_x, top_pole_y), fill=pole_col, width=pole_w)  # draw pole

        # --------------- Render HUD --------------------
        font = ImageFont.truetype("arial.ttf", 16)


        # draw.text((100, 200), str(action_val), font=font, fill=(255, 0, 0, 255))  # Left/right action
        # draw.text((0, 100), 'x_pos {:0.4f}\nstep     {}'
        #           .format(obs[0], step),
        #           font=font, fill=(150, 150, 150, 255))

        draw.line((cart_x, cart_y - cart_h // 2, top_pole_x, top_pole_y), fill=pole_col, width=pole_w)

        if action_val == 1:
            cart_p1 = (cart_x - cart_w // 2, cart_y - cart_h // 3)
            cart_p2 = (cart_x - cart_w // 2, cart_y + cart_h // 3)
            arrow_point = (cart_x - cart_w // 1.3, cart_y)
            draw.polygon([cart_p1, cart_p2, arrow_point], fill=(255, 0, 0))  # draw thrust arrow left
        else:
            cart_p3 = (cart_x + cart_w // 2, cart_y - cart_h // 3)
            cart_p4 = (cart_x + cart_w // 2, cart_y + cart_h // 3)
            arrow_point_r = (cart_x + cart_w // 1.3, cart_y)
            draw.polygon([cart_p3, cart_p4, arrow_point_r], fill=(255, 0, 0))  # draw thrust arrow right


        draw.rectangle((cart_x - cart_w // 2, cart_y - cart_h // 2, cart_x + cart_w // 2, cart_y + cart_h // 2),
                       fill=cart_col)  # draw cart
        draw.text((cart_x - cart_w // 2 + 22 - 4 * len(str(step)), 370), str(step), font=font,
                  fill=(255, 255, 255, 255))

        return np.array(img)



In [9]:

import numpy as np
# from render_utils import *
import math


def discount_rewards(rewards, discount_rate, obs):
    """ Modify reward to improve historic behaviour """

    discounted_rewards = np.zeros(len(rewards))
    cumulative_rewards = 0
    phi = 1.0

    # print('step, obs[step], rewards[step], drift_penalty, cumulative_rewards, discounted_rewards[step]')

    for step in reversed(range(len(rewards))):
        # Prevent DRIFT by adding penalty (distance from centre)
        drift_penalty = phi * abs(obs[step][0])

        # drift_penalty = max(0.2, phi * abs(obs[step][0]))
        # drift_penalty = 0.5 if abs(obs[step][0]) > 0.3 else 0

        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate - drift_penalty
        discounted_rewards[step] = cumulative_rewards
        # print(step, obs[step], rewards[step], drift_penalty, cumulative_rewards, discounted_rewards[step])

    return discounted_rewards


def discount_and_normalize_rewards(all_rewards, all_obs, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate, all_obs[i])
                              for i, rewards in enumerate(all_rewards)]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]


"""
New reward function:
    Add penalty for (-theta) and (+momentum)
    or the inverse: (+theta) and (-momentum)
"""

'\nNew reward function:\n    Add penalty for (-theta) and (+momentum)\n    or the inverse: (+theta) and (-momentum)\n'

In [10]:
# ---------- Initialise CartePole -------------
env = gym.make('CartPole-v0')
obs = env.reset()
obs

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


array([-0.03861935, -0.00967731,  0.00170503, -0.0040774 ])

* `obs` is the new observation 
        [position of cart, velocity of cart, angle of pole, rotation rate of pole]
* `reward` is given 1 point at every step, the goal is to keep running as long as possible
* `done` This value will be true when the episode is over
* `info` This dictionary may provide extra debug information in other environments

The episode ends when the pole is more than 15 degrees from vertical, or the cart moves more than 2.4 units from the center.

In [0]:
# Helper functions

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures and animations
%matplotlib nbagg
import matplotlib
import matplotlib.animation as animation
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rl"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

### Reinforcement Learning - Policy Gradients
* The Policy Gradients algorithm tackles the problem by playing multiple games

In [0]:
# ------- Build Tensorflow Graph -------

import tensorflow as tf

reset_graph()

n_inputs = 4
n_hidden = 12
n_outputs = 1

learning_rate = 0.05

initializer = tf.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape=[None, n_inputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs)
outputs = tf.nn.sigmoid(logits)  # probability of action 0 (left)
p_left_and_right = tf.concat(axis=1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples=1)

y = 1. - tf.to_float(action)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradient_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradient_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

# --------------------------------------

In [0]:
def render_policy_net(model_path, X, n_max_steps = 1000):
    frames = []
    env = gym.make("CartPole-v0")
    obs = env.reset()
    action_val = 0
    
    with tf.Session() as sess:
        saver.restore(sess, model_path)
        for step in range(n_max_steps):
            # Send step to rendering function
            img = render_cart_pole(env, obs, step, action_val)
            frames.append(img)
            
            action_val = action.eval(feed_dict={X: obs.reshape(1, n_inputs)})
            obs, reward, done, info = env.step(action_val[0][0])
            
            if done:
                break
    env.close()
    return frames

In [0]:
# ------- Run Session -------

env = gym.make("CartPole-v0")

n_games_per_update = 10
n_max_steps = 1000
n_iterations = 5000
save_iterations = 10
discount_rate = 0.95

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        print("\rIteration: {}".format(iteration), end="")
        all_rewards = []
        all_gradients = []
        all_obs = []
        for game in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            current_obs = []
            obs = env.reset()
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run([action, gradients], feed_dict={X: obs.reshape(1, n_inputs)})
                obs, reward, done, info = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                current_obs.append(obs)
                if done:
                    break
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
            all_obs.append(current_obs)

        all_rewards = discount_and_normalize_rewards(all_rewards, all_obs, discount_rate=discount_rate)
        feed_dict = {}
        for var_index, gradient_placeholder in enumerate(gradient_placeholders):
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                      for game_index, rewards in enumerate(all_rewards)
                                          for step, reward in enumerate(rewards)], axis=0)
            feed_dict[gradient_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            saver.save(sess, "./models/policy-grad-drift-penalty_home.ckpt")
            print(' Avg. Frames:', np.mean([len(reward) for reward in all_rewards]))
            if np.mean([len(reward) for reward in all_rewards]) > 195: # early stopping
                break    

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Iteration: 0 Avg. Frames: 16.0
Iteration: 10 Avg. Frames: 64.3
Iteration: 20 Avg. Frames: 98.2
Iteration: 28

In [0]:
env.close()

In [47]:
frames = render_policy_net("models/policy-grad-drift-penalty_home.ckpt", X, n_max_steps=2000)
video = plot_animation(frames, repeat=True)
plt.show()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from models/policy-grad-drift-penalty_home.ckpt


NameError: ignored

In [24]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# PyDrive reference:
# https://googledrive.github.io/PyDrive/docs/build/html/index.html

# 2. Create & upload a file text file.
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

# 3. Load a file by ID and print its contents.
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 1nH0xir1orPj1HJl6Z0kiQZ0S2oi7Wxk_
Downloaded content "Sample upload file content"
